In [72]:
import pandas as pd
import re
import json
import time
import requests
from requests.auth import HTTPBasicAuth
from elasticsearch import Elasticsearch
import elasticsearch
from elasticsearch.helpers import bulk
from dotenv import load_dotenv
load_dotenv()
import os

print(elasticsearch.__version__)

(7, 13, 3)


In [73]:
# !!! CUSTOMIZE THIS SECTION WITH YOUR CREDENTIALS !!!

USER = 'elastic'
# PWD = os.getenv("ELASTIC_PASSWORD")
PWD = 'asdqweasd'
index_name = 'books'
ES_ENDPOINT = 'https://localhost:9200'

path_to_ca_certificates = '../certs/ca/ca.crt'

### Read data

In [74]:
df = pd.read_csv('books.csv')
df = df.set_index('id')
df.head()

,author_name,title,country,language,year
id,,,,,
14602826,"Yearsley, Ann",Poems on several occasions,England,English,1786
14602830,"A, T.",A Satyr against Vertue. (A poem: supposed to b...,England,English,1679
14602831,"A, T.","The Aeronaut, a poem; founded almost entirely,...",Ireland,English,1816
14602832,"Albert, Prince Consort, consort of Victoria, Q...","The Prince Albert, a poem",Ireland,English,1868
14602833,"Anslow, Robert","The Defeat of the Spanish Armada, A.D. 1588. A...",England,English,1888


In [75]:
#transform dataframe into json format
docs = df.to_dict(orient='records')
doc_ids = df.index
print(doc_ids[0])
print(docs[0])

14602826
{'author_name': 'Yearsley, Ann', 'title': 'Poems on several occasions', 'country': 'England', 'language': 'English', 'year': 1786}



### Elasticsearch Python wrapper

In [76]:
def create_index(es, index_name, settings=None):
    """
    Create an Elasticsearch index
    @param es: an Elasticsearch object
    @param index_name: the name of the new index to be created
    @param settings: the index settings
    @return whether the index was created
    """
    is_created = False
    try:
        if es.indices.exists(index=[index_name]):
            es.indices.delete(index=[index_name], ignore=[404])
        es.indices.create(index=index_name)
        is_created = True
    except Exception as ex:
        print(str(ex))
    return is_created

In [77]:
# Index settings
settings_basic = {
        "number_of_shards": 4,
        "number_of_replicas": 2,
        "analysis": {
            "analyzer": {"std_english": {"type": "standard", "stopwords": "_english_" }}
        }
}

In [78]:
#connect to the local elasticsearch node and authenticate
# print(f'{USER}, {PWD}') 
es = Elasticsearch(hosts=[ES_ENDPOINT], ca_certs=path_to_ca_certificates, http_auth=(USER, PWD))

#create an index
is_created = create_index(es, index_name, settings=settings_basic)
print(f'Index creation: {is_created}')

Index creation: True


In [79]:
#loops over the first 10 documents
for i, doc in zip(doc_ids[0:10], docs[0:10]):
    #index the documents with corresponding ids
    res = es.index(index=index_name, id=i, body=doc)
    print(res)
# see also the bulk functions for importing under: elasticsearch.helpers

{'_index': 'books', '_id': '14602826', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
{'_index': 'books', '_id': '14602830', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
{'_index': 'books', '_id': '14602831', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}
{'_index': 'books', '_id': '14602832', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 3, '_primary_term': 1}
{'_index': 'books', '_id': '14602833', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1}
{'_index': 'books', '_id': '14602834', '_version': 1, 'result': 'created', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 5, '_primary_term': 1}
{'_index': 'book

In [80]:
actions = [
  {
    "_index": index_name,
    "_id": doc_id,
    "_source": doc
  }
  for doc_id, doc in list(zip(doc_ids, docs))
]

# send actions in bulk (the API takes care of chunking them optimally)
bulk(es, actions)

(52695, [])

In [81]:
# get settings info of the selected index
es.indices.get_settings(index=index_name)

{'books': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '1',
    'provided_name': 'books',
    'creation_date': '1676710670269',
    'number_of_replicas': '1',
    'uuid': 'F5JjEnGaTLGdOVmOINWwmw',
    'version': {'created': '8050399'}}}}}

In [82]:
# retrieve a document with a given ID
es.get(index=index_name, id=doc_ids[1])

{'_index': 'books',
 '_id': '14602830',
 '_version': 2,
 '_seq_no': 11,
 '_primary_term': 1,
 'found': True,
 '_source': {'author_name': 'A, T.',
  'title': 'A Satyr against Vertue. (A poem: supposed to be spoken by a Town-Hector )',
  'country': 'England',
  'language': 'English',
  'year': 1679}}

In [83]:
# this is how you would delete the index
es.indices.delete(index=index_name, ignore=404)

{'acknowledged': True}

### Elasticsearch with python cURL (Requests)

In [84]:
class Elastic:
    """
    A convenience object to send HTTP requests to Elasticsearch
    """
    def __init__(self, endpoint, username, password, path_to_ca_certificates):
        """
        @param endpoint: the URL of the Elasticsearch instance
        @param username: the Elasticsearch username 
        @param password: the Elasticsearch password
        """
        self.header = {'Content-Type': 'application/json', 'charset':'UTF-8'}
        #self.header={'Content-Type': '--data-binary application/x-ndjson'}
        self.endpoint = endpoint
        self.username = username
        self.password = password
        self.path_to_ca_certificates = path_to_ca_certificates
        self.methods_mapping = {'get': requests.get, 
                                'put':requests.put, 
                                'post':requests.post, 
                                'delete':requests.delete}
        
    def curl(self, method, handle, json=None):
        """
        Sends an HTTP request to the Elasticsearch instance
        @param method: can be 'get', 'put', 'post', 'delete'
        @param handle: the API handle to be appended to the Elasticsearch url
        @param json: the json payload of the HTTP request
        """
        http_method = self.methods_mapping[method.lower()]
        r = http_method(f'{self.endpoint}/{handle}', auth=HTTPBasicAuth(USER, PWD), 
                        headers=self.header, json=json,
                        verify = self.path_to_ca_certificates)
        return r

In [85]:
e = Elastic(ES_ENDPOINT, USER, PWD, path_to_ca_certificates)

In [86]:
# create another index jsut as an example. in the following, we will keep using the book index
# created using the Elasticsearch API

create_index_json={
  "mappings" : {
      "properties" : {
        "author_name" : {
          "type" : "text"
        },
        "country" : {
          "type" : "keyword"
        },
        "language" : {
          "type" : "keyword"
        },
        "title" : {
          "type" : "text"
        },
        "year" : {
          "type" : "long"
        }
      }
  },
  "settings": {
    "number_of_shards": 4, 
    "number_of_replicas": 2, 
    "index.max_result_window": 20000,
    "index" : {
        "similarity" : {
          "default" : {
            "type" : "BM25", "b": 0.75, "k1": 1.2
          }
        }
    },
    "analysis": {
      "analyzer": {
        "std_english": {"type": "standard", "stopwords": "_english_" }
      }
    }
  }
}

# create an index
r = e.curl('put', index_name, json=create_index_json)
r.json()

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'books'}

In [87]:
# get the index details and settings
r = e.curl('get', index_name)
r.json()

{'books': {'aliases': {},
  'mappings': {'properties': {'author_name': {'type': 'text'},
    'country': {'type': 'keyword'},
    'language': {'type': 'keyword'},
    'title': {'type': 'text'},
    'year': {'type': 'long'}}},
  'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '4',
    'provided_name': 'books',
    'similarity': {'default': {'type': 'BM25', 'b': '0.75', 'k1': '1.2'}},
    'max_result_window': '20000',
    'creation_date': '1676710679022',
    'analysis': {'analyzer': {'std_english': {'type': 'standard',
       'stopwords': '_english_'}}},
    'number_of_replicas': '2',
    'uuid': 's3j-119ORDin7yv1zi0DVw',
    'version': {'created': '8050399'}}}}}

In [88]:
# deactivate refresh in preparation of data indexing
r = e.curl('put', 'books/_settings', {'index' : {'refresh_interval' : -1}})
r.json()

{'acknowledged': True}

In [89]:
# index documents with their individual ids (use bulk below for speedup)
for doc_id, doc in list(zip(doc_ids, docs))[0:10]:
    r = e.curl('post', f'books/_doc/{doc_id}', json=doc)
r.json()

{'_index': 'books',
 '_id': '14602838',
 '_version': 1,
 'result': 'created',
 '_shards': {'total': 3, 'successful': 1, 'failed': 0},
 '_seq_no': 2,
 '_primary_term': 1}

In [90]:
# bulk indexing (via official API)

#connect to the local elasticsearch node and authenticate
es = Elasticsearch([ES_ENDPOINT], ca_certs=path_to_ca_certificates, http_auth=(USER, PWD))

actions = [
  {
    "_index": index_name,
    "_id": doc_id,
    "_source": doc
  }
  for doc_id, doc in list(zip(doc_ids, docs))
]

# send actions in bulk (the API takes care of chunking them optimally)
bulk(es, actions)

(52695, [])

In [91]:
# reset the refresh interval to 2 seconds
r = e.curl('put', 'books/_settings', {'index' : {'refresh_interval' : '2s'}})
r.json()

{'acknowledged': True}

In [92]:
r = e.curl('get', f'books/_doc/{doc_ids[42]}')
r.json()

{'_index': 'books',
 '_id': '14602871',
 '_version': 1,
 '_seq_no': 12,
 '_primary_term': 1,
 'found': True,
 '_source': {'author_name': 'Smedley, Edward, Fellow of Sidney Sussex College, Cambridge',
  'title': 'The Death of Saul and Jonathan. A poem',
  'country': 'England',
  'language': 'English',
  'year': 1814}}

In [93]:
# this is how you would delete the index
# r = e.curl('delete', 'books')
# r.json()

### Search queries [EXERCISES]

#### Empty query

In [94]:
# empty query
r = e.curl('get', f'books/_search')
r.json()

{'took': 16,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 1.0,
  'hits': [{'_index': 'books',
    '_id': '14602831',
    '_score': 1.0,
    '_source': {'author_name': 'A, T.',
     'title': 'The Aeronaut, a poem; founded almost entirely, upon a statement, printed in the newspapers, of a voyage from Dublin, in October, 1812',
     'country': 'Ireland',
     'language': 'English',
     'year': 1816}},
   {'_index': 'books',
    '_id': '14602832',
    '_score': 1.0,
    '_source': {'author_name': 'Albert, Prince Consort, consort of Victoria, Queen of Great Britain',
     'title': 'The Prince Albert, a poem',
     'country': 'Ireland',
     'language': 'English',
     'year': 1868}},
   {'_index': 'books',
    '_id': '14602838',
    '_score': 1.0,
    '_source': {'author_name': 'Brabant, Henry, Sir',
     'title': 'The Eve of the Revolution; in Newcastle-upon-Tyne. (The Case

#### Aggregation query

In [ ]:
""" 
EXERCISE A: 
execute an aggregation query to count the number of books writte in each country
"""

query = {
    "aggregations": {
        "by_category": {
            "terms": {
                "field": "country",
                "size":100
            }   
        }
    }
}
r = e.curl('get', f'books/_search', query)
r.json()

#### Full-text query

In [113]:
""" 
EXERCISE B: 
execute a full-text query for the query "love magic"
"""
query = {
    "query": {
        "match": {
            "title": {
                "query": "love magic"
            }
        }
    }
}
r = e.curl('get', f'books/_search', query)
r.json()

{'took': 5,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 456, 'relation': 'eq'},
  'max_score': 15.985714,
  'hits': [{'_index': 'books',
    '_id': '14812010',
    '_score': 15.985714,
    '_source': {'author_name': 'Grant, Forrest, Mrs',
     'title': 'The Magic of Love. A novel',
     'country': 'England',
     'language': 'English',
     'year': 1874}},
   {'_index': 'books',
    '_id': '14628808',
    '_score': 14.215486,
    '_source': {'author_name': 'Dorisi, Lisa',
     'title': 'The Enchanter; or, Love and Magic. A Musical Drama',
     'country': 'England',
     'language': 'English',
     'year': 1760}},
   {'_index': 'books',
    '_id': '14816800',
    '_score': 12.649069,
    '_source': {'author_name': 'Lansdowne, George Granville, Baron',
     'title': 'Poems upon Several Occasions. (The British Enchanters; or, No magic like love.)',
     'country': 'England',
     'language': 'English',
     'year'

#### Exact match query

In [114]:
""" 
EXERCISE C: 
execute an exact-match query for the query "magic love"
"""

# exact match
query = {
    "query": {
        "match_phrase": {
            "title": {
                "query": "magic love"
            }
        }
    }
}
r = e.curl('get', f'books/_search', query)
r.json()

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

#### Multi-field full-text query with field boosting

In [121]:
""" 
EXERCISE D: 
Execute a query that searches both on tile and author
Weights the importance of matches on the author field twice as much as matches on the title
Sse "shakespeare" as query term
"""

query_term = "shakespeare"

# field boosting in the form of queryfield^x
query = {
    "query": {
        "multi_match": {
            "query": query_term,
            "fields": ["title", "author_name^2"],
            "type": "phrase"
        }
    }
}

r = e.curl('get', f'books/_search', query)
r.json()

{'took': 4,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 95, 'relation': 'eq'},
  'max_score': 16.698162,
  'hits': [{'_index': 'books',
    '_id': '14628923',
    '_score': 16.698162,
    '_source': {'author_name': 'Shakespeare, William',
     'title': 'A Midsummer Nights Dream : With alterations and additions and several new songs  . As it is performed at the Theatre-Royal in Drury-Lane',
     'country': 'England',
     'language': 'English',
     'year': 1763}},
   {'_index': 'books',
    '_id': '14824041',
    '_score': 16.698162,
    '_source': {'author_name': 'Shakespeare, William',
     'title': 'The plays of William Shakspeare. In ten volumes. With the corrections and illustrations of various commentators; to which are added notes by S. Johnson and G. Steevens. (An attempt to ascertain the order in which the plays attributed to Shakespeare were written, by E. Malone.) The second edition, revised and augm

#### Combining different queries

In [126]:
"""
EXERCISE E:
execute a *single* boolean query returning books that:
- have the "queen mary" in the title and were written in England
- should NOT have been published in the range of years [1850-1913]
"""

query = {
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "title": "queen mary"
                    }
                },
                {
                    "match": {
                        "country": "england"
                    }
                },
            ]
        }
    }
}

r = e.curl('get', f'books/_search', query)
r.json()

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

#### Fuzzy queries

In [100]:
"""
EXERCISE F:
execute a fuzzy query for the query "comander" with at most 50 expansions 
and considering transpositions
"""

'\nEXERCISE F:\nexecute a fuzzy query for the query "comander" with at most 50 expansions \nand considering transpositions\n'

#### Get TF of terms in document

In [101]:
"""
EXERCISE G:
get the term frequencies (in the title) of document with id = 4200
consider only words with a minimum length of 4 and a minimum term frequency of 2
"""

'\nEXERCISE G:\nget the term frequencies (in the title) of document with id = 4200\nconsider only words with a minimum length of 4 and a minimum term frequency of 2\n'

### Solutions

In [102]:
# SOLUTION TO EXERCISE A

# aggregation query
query = {
    "aggregations": {
        "by_category": {
            "terms": {
                "field": "country",
                "size":100
            }
        }
    }
}
r = e.curl('get', f'books/_search', query)
r.json()

{'took': 14,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 1.0,
  'hits': [{'_index': 'books',
    '_id': '14602831',
    '_score': 1.0,
    '_source': {'author_name': 'A, T.',
     'title': 'The Aeronaut, a poem; founded almost entirely, upon a statement, printed in the newspapers, of a voyage from Dublin, in October, 1812',
     'country': 'Ireland',
     'language': 'English',
     'year': 1816}},
   {'_index': 'books',
    '_id': '14602832',
    '_score': 1.0,
    '_source': {'author_name': 'Albert, Prince Consort, consort of Victoria, Queen of Great Britain',
     'title': 'The Prince Albert, a poem',
     'country': 'Ireland',
     'language': 'English',
     'year': 1868}},
   {'_index': 'books',
    '_id': '14602838',
    '_score': 1.0,
    '_source': {'author_name': 'Brabant, Henry, Sir',
     'title': 'The Eve of the Revolution; in Newcastle-upon-Tyne. (The Case

In [111]:
# SOLUTION TO EXERCISE B

# full-text query
query = {
    "query": {
        "match": {
            "title": {
                "query": "love magic"
            }
        }
    }
}
r = e.curl('get', f'books/_search', query)
r.json()

{'took': 10,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 456, 'relation': 'eq'},
  'max_score': 15.985714,
  'hits': [{'_index': 'books',
    '_id': '14812010',
    '_score': 15.985714,
    '_source': {'author_name': 'Grant, Forrest, Mrs',
     'title': 'The Magic of Love. A novel',
     'country': 'England',
     'language': 'English',
     'year': 1874}},
   {'_index': 'books',
    '_id': '14628808',
    '_score': 14.215486,
    '_source': {'author_name': 'Dorisi, Lisa',
     'title': 'The Enchanter; or, Love and Magic. A Musical Drama',
     'country': 'England',
     'language': 'English',
     'year': 1760}},
   {'_index': 'books',
    '_id': '14816800',
    '_score': 12.649069,
    '_source': {'author_name': 'Lansdowne, George Granville, Baron',
     'title': 'Poems upon Several Occasions. (The British Enchanters; or, No magic like love.)',
     'country': 'England',
     'language': 'English',
     'year

In [104]:
# SOLUTION TO EXERCISE C

# exact match query
query = {
  "query": {
    "match_phrase": {
      "title": {
        "query": "magic love"
      }
    }
  }
}
r = e.curl('get', f'books/_search', query)
r.json()

{'took': 18,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [105]:
# SOLUTION TO EXERCISE D

# Full text query, multiple fields with boosting
query = {
"query": {
    "multi_match": {
            "query": "shakespeare",
            "fields": ["title", "author_name^2"],
            "type": "phrase"
        }
    }
}
r = e.curl('get', f'books/_search', query)
r.json()

{'took': 17,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 95, 'relation': 'eq'},
  'max_score': 16.698162,
  'hits': [{'_index': 'books',
    '_id': '14628923',
    '_score': 16.698162,
    '_source': {'author_name': 'Shakespeare, William',
     'title': 'A Midsummer Nights Dream : With alterations and additions and several new songs  . As it is performed at the Theatre-Royal in Drury-Lane',
     'country': 'England',
     'language': 'English',
     'year': 1763}},
   {'_index': 'books',
    '_id': '14824041',
    '_score': 16.698162,
    '_source': {'author_name': 'Shakespeare, William',
     'title': 'The plays of William Shakspeare. In ten volumes. With the corrections and illustrations of various commentators; to which are added notes by S. Johnson and G. Steevens. (An attempt to ascertain the order in which the plays attributed to Shakespeare were written, by E. Malone.) The second edition, revised and aug

In [106]:
# SOLUTION TO EXERCISE E

# One or more queries can be specified in each of the clauses
# All the clauses are optional
# MUST: A document must match all of the queries
# MUST_NOT: A document must not match any of the queries
# SHOULD: A document does not have to match the queries, but it is considered more relevant if it does
# FILTER: Filters with yes/no categories
query = {
  "size" : 100,
  "query": {
    "bool": {
      "must": [
        {
          "match": {
                "title": "queen mary"
          }
        },
        {
          "match": {
              "country": "England"
          }
        }
      ],
      "must_not": [
        {
          "range": {
            "year": {
                "gte": 1850,
                "lte": 1913
            }
          }
        }
      ]
    }
  }
}
r = e.curl('get', f'books/_search', query)
r.json()

{'took': 26,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 104, 'relation': 'eq'},
  'max_score': 14.339016,
  'hits': [{'_index': 'books',
    '_id': '14828855',
    '_score': 14.339016,
    '_source': {'author_name': 'Saint John, John, Honourable',
     'title': 'Mary Queen of Scots, a tragedy',
     'country': 'England',
     'language': 'English',
     'year': 1789}},
   {'_index': 'books',
    '_id': '14866478',
    '_score': 14.200716,
    '_source': {'author_name': 'Buckingham, Leicester Silk',
     'title': 'Memoirs of Mary Stuart, Queen of Scotland',
     'country': 'England',
     'language': 'English',
     'year': 1844}},
   {'_index': 'books',
    '_id': '14816241',
    '_score': 11.958086,
    '_source': {'author_name': 'Banks, John',
     'title': 'The Albion Queens, or, The Death of Mary Queen of Scotland',
     'country': 'England',
     'language': 'English',
     'year': 1704}},
   {'_index': '

In [107]:
# SOLUTION TO EXERCISE F

query ={
  "query": {
    "fuzzy": {
      "title": {
        "value": "comander",
        "fuzziness": "AUTO",
        "max_expansions": 50,
        "prefix_length": 0,
        "transpositions": True,
        "rewrite": "constant_score"
      }
    }
  }
}
r = e.curl('get', f'books/_search', query)
r.json()

{'took': 69,
 'timed_out': False,
 '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 64, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'books',
    '_id': '14603267',
    '_score': 1.0,
    '_source': {'author_name': 'Pindar, Peter, Jun, pseudonym [i.e. John Agg]',
     'title': 'Three R----l Bloods; or, a Lame r-t, a darling commander, and a love-sick admiral. A poem. By Peter Pindar, Jun. (Seventh edition.)',
     'country': 'England',
     'language': 'English',
     'year': 1812}},
   {'_index': 'books',
    '_id': '14809925',
    '_score': 1.0,
    '_source': {'author_name': 'Elliott, Robert, Commander, R.N.',
     'title': 'Views in India, China, and on the Shores of the Red Sea: drawn by Prout, Stanfield, Cattermole, Purser, Cox, Austen, &c. from original sketches by Commander R. Elliott ... With descriptions by Emma Roberts',
     'country': 'England',
     'language': 'English',
     'year': 1835}},
   {'_index': '

In [108]:
# SOLUTION TO EXERCISE G

# https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-termvectors.html
query = {
    "fields" : ["title"],
    "term_statistics" : True,
    "field_statistics" : True,
    "positions": True,
    "filter": {
        "min_word_length": 4,
        "min_term_freq" : 2
  }
}
#r = e.curl('get', f'books/_doc/{doc_ids[4200]}')
r = e.curl('get', f'books/_termvectors/{doc_ids[4200]}', query)
r.json()

{'_index': 'books',
 '_id': '14767985',
 '_version': 1,
 'found': True,
 'took': 41,
 'term_vectors': {'title': {'field_statistics': {'sum_doc_freq': 148369,
    'doc_count': 13219,
    'sum_ttf': 166533},
   'terms': {'with': {'doc_freq': 1908,
     'ttf': 2076,
     'term_freq': 2,
     'tokens': [{'position': 9, 'start_offset': 52, 'end_offset': 56},
      {'position': 24, 'start_offset': 135, 'end_offset': 139}],
     'score': 5.8703027}}}}}